Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)  

In [2]:
pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [4]:
from pyspark import SparkFiles

covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

In [5]:
from pyspark.sql import functions as F

In [6]:
ITOG = (df
  .select('iso_code', 'location', F.round(F.col('total_cases').cast('integer') / F.col('population').cast('integer')*100,2)
  .alias('процент переболевших')).filter(F.col('date').isin('2021-03-31', '2021-03-31'))
  .orderBy(F.col('процент переболевших').desc())
  .show(15))

+--------+-------------+--------------------+
|iso_code|     location|процент переболевших|
+--------+-------------+--------------------+
|     AND|      Andorra|               15.54|
|     MNE|   Montenegro|               14.52|
|     CZE|      Czechia|               14.31|
|     SMR|   San Marino|               13.94|
|     SVN|     Slovenia|               10.37|
|     LUX|   Luxembourg|                9.85|
|     ISR|       Israel|                9.63|
|     USA|United States|                 9.2|
|     SRB|       Serbia|                8.83|
|     BHR|      Bahrain|                8.49|
|     PAN|       Panama|                8.23|
|     PRT|     Portugal|                8.06|
|     EST|      Estonia|                8.02|
|     SWE|       Sweden|                7.97|
|     LTU|    Lithuania|                7.94|
+--------+-------------+--------------------+
only showing top 15 rows



Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [7]:
from pyspark.sql import Window

In [8]:
res0 = df.select(F.col("date").alias("Число"),
                 F.col("location").alias("Страна"),
                 F.col("new_cases").alias("кол-во новых случаев")).where(df[3].between("2021-03-25", "2021-03-31")).groupBy("Страна","Число").agg(F.sum("кол-во новых случаев").alias("кол-во новых случаев"))

In [9]:
res0.show()

+--------------------+----------+--------------------+
|              Страна|     Число|кол-во новых случаев|
+--------------------+----------+--------------------+
|              Africa|2021-03-25|             12490.0|
|             Algeria|2021-03-28|                86.0|
|           Australia|2021-03-26|                13.0|
|          Bangladesh|2021-03-31|              5358.0|
|             Comoros|2021-03-25|                 5.0|
|              Cyprus|2021-03-31|               434.0|
|                Iraq|2021-03-27|              5062.0|
|             Armenia|2021-03-28|               777.0|
|               Benin|2021-03-31|                 0.0|
|            Cameroon|2021-03-27|                 0.0|
|                Cuba|2021-03-27|               950.0|
|             Finland|2021-03-31|               607.0|
|              Guinea|2021-03-31|               115.0|
|             Iceland|2021-03-29|                20.0|
|             Lebanon|2021-03-30|              2668.0|
|         

In [24]:
#оконная функция
Specs = Window.partitionBy("Страна").orderBy(F.col("кол-во новых случаев").desc())
res1 = res0.withColumn("№", F.row_number().over(Specs))


In [25]:
#Итого с удалением обобщающих названий
res1.filter(F.col("№") == 1).filter(~F.col("Страна").isin("World", "Europe", "European Union", "Asia","South America","North America", "United States")).select("Число", "Страна", "кол-во новых случаев").orderBy(F.col("кол-во новых случаев").desc()).show()

+----------+-----------+--------------------+
|     Число|     Страна|кол-во новых случаев|
+----------+-----------+--------------------+
|2021-03-25|     Brazil|            100158.0|
|2021-03-31|      India|             72330.0|
|2021-03-31|     France|             59054.0|
|2021-03-31|     Turkey|             39302.0|
|2021-03-26|     Poland|             35145.0|
|2021-03-31|    Germany|             25014.0|
|2021-03-26|      Italy|             24076.0|
|2021-03-25|       Peru|             19206.0|
|2021-03-26|     Africa|             18544.0|
|2021-03-26|    Ukraine|             18226.0|
|2021-03-30|     Sweden|             16427.0|
|2021-03-31|  Argentina|             16056.0|
|2021-03-29|      Spain|             15501.0|
|2021-03-26|    Hungary|             11265.0|
|2021-03-31|       Iran|             10330.0|
|2021-03-29|Philippines|             10002.0|
|2021-03-30|     Serbia|              9983.0|
|2021-03-27|Netherlands|              9160.0|
|2021-03-25|     Russia|          

Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [51]:
res2 = df.select(F.col("date").alias("Число"),
                 F.col("new_cases").alias("Кол-во новых случаев вчера")).where(df[3].between("2021-03-25", "2021-03-31")).where(df[2] == "Russia")

In [52]:
# окошко
specs = Window.orderBy(F.col("Число"))
res2 = res2.withColumn("Кол-во новых случаев сегодня", F.lag(F.col("Кол-во новых случаев вчера")).over(specs))
res2.show()

+----------+--------------------------+----------------------------+
|     Число|Кол-во новых случаев вчера|Кол-во новых случаев сегодня|
+----------+--------------------------+----------------------------+
|2021-03-25|                    9128.0|                        NULL|
|2021-03-26|                    9073.0|                      9128.0|
|2021-03-27|                    8783.0|                      9073.0|
|2021-03-28|                    8979.0|                      8783.0|
|2021-03-29|                    8589.0|                      8979.0|
|2021-03-30|                    8162.0|                      8589.0|
|2021-03-31|                    8156.0|                      8162.0|
+----------+--------------------------+----------------------------+



In [53]:
res2.select("Число", "Кол-во новых случаев вчера", "Кол-во новых случаев сегодня",
            (F.col("Кол-во новых случаев вчера") - F.col("Кол-во новых случаев сегодня")).alias("Дельта")
            ).where(F.col("Кол-во новых случаев сегодня").isNotNull()).orderBy(F.col("Число").desc()).show()


+----------+--------------------------+----------------------------+------+
|     Число|Кол-во новых случаев вчера|Кол-во новых случаев сегодня|Дельта|
+----------+--------------------------+----------------------------+------+
|2021-03-31|                    8156.0|                      8162.0|  -6.0|
|2021-03-30|                    8162.0|                      8589.0|-427.0|
|2021-03-29|                    8589.0|                      8979.0|-390.0|
|2021-03-28|                    8979.0|                      8783.0| 196.0|
|2021-03-27|                    8783.0|                      9073.0|-290.0|
|2021-03-26|                    9073.0|                      9128.0| -55.0|
+----------+--------------------------+----------------------------+------+

